In [369]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import integrate
import scipy.linalg

Ejercicios: Álgebra lineal

Problemas: 3,4,5,6,7,9,10,12,13,14,15

Listos: 3,6,7,9,  ,15

## Punto 3. Algoritmo para mutiplicar dos matrices

In [370]:
def MultiplyMatrix(matrix1, matrix2):
    
    if len(matrix1[0])!=len(matrix2):
        return 'No es posible realizar el producto.'
    else:
        resultMatrix = np.zeros([len(matrix1),len(matrix2[0])])
        for i in range(len(matrix1)):
            for j in range(len(matrix2[0])):
                for k in range(len(matrix2)):
                    resultMatrix[i][j] += matrix1[i][k]*matrix2[k][j]
        return resultMatrix

In [371]:
M1 = np.array([[1, 0, 0],[5, 1, 0],[-2, 3, 1]])
M2 = np.array([[4, -2, 1],[0, 3, 7],[0, 0, 2]])

MultiplyMatrix(M1,M2)

array([[ 4., -2.,  1.],
       [20., -7., 12.],
       [-8., 13., 21.]])

## Punto 4. 
**(Theoretical)** Muestre con detalle que la sustituci´on hacia adelante se expresa como:
$$x_i = b_i + \sum_{j=0}^{i-1} A_{ij}x_j.$$

Esta prueba será realizada por inducción en $i$:

**Caso Base.**

Se tiene que $i=1$. Entonces, $$x_1 = b_1,$$ que corresponde a la primera ecuación del sistema matricial.

**Hipótesis de Inducción.**

Se toma como cierto el caso de $i-1$:
$$x_{i-1} = b_{i-1} + \sum_{j=0}^{i-2} A_{ij}x_j.$$

**Paso Inductivo.**

Para el paso inductivo se supondrá la hipótesis de inducción y se demostrará que es cierto para $i$. Para el sistema $i$ se tiene que


$$ b_{i} = \sum_{j=0}^{i-1} A_{ij}x_i = A_{ii}x_i + \sum_{j=0}^{i-1} A_{ij}x_i$$


## Punto 6. 

In [372]:
def SOR(A, b, x0, omega, itmax=800, tol=1e-9):
    
    x = x0.copy()
    
    for it in range(1,itmax):
        for i in range(A.shape[0]):
            sum_ = 0.
            for j in range(A.shape[1]):
                if j!=i:
                    sum_ += A[i,j]*x[j]
            x[i] = (1-omega)*x[i] + omega*((b[i] - sum_)/A[i,i])
        
        residuo = np.linalg.norm( np.dot(A,x) - b )
        
        if np.allclose(residuo, tol, atol=tol):
            break
    return x, it



In [475]:
A = np.array([[3,-1,-1],[-1.,3.,1.],[2,1,4]])
b = np.array([1.,3.,7.])
x = np.array([0.,0.,0.])

omega = np.linspace(0.1, 2, 100)
omega_opt = np.inf
iter_opt = 1000-1

for i in omega:
    if SOR(A, b, x, i)[1]< (iter_opt):
        omega_opt = i
        iter_opt = SOR(A, b, x, i)[1]
        X = SOR(A, b, x, i)[0]
        
print('Solución: ',X, '\nOmega óptimo:', omega_opt, '\nNúmero óptimo de iteraciones:', iter_opt)

Solución:  [1. 1. 1.] 
Omega óptimo: 0.9828282828282827 
Número óptimo de iteraciones: 15


## Punto 7. Algoritmo de descomposición

In [374]:
def LUdecomposition(matrix):
    
    n = len(matrix)
    L = np.zeros((n,n))
    U = np.zeros((n,n))

    for i in range(n):
        L[i][i] = 1
        # Upper matrix
        for j in range(i, n):
            for k in range(i):
                U[i][j] = matrix[i][j] - np.sum(U[k][j]*L[i][k])
        # Lower matrix
        for j in range(i + 1, n):
            for k in range(i):
                L[j][i] = (matrix[j][i] - np.sum(U[k][i] * L[j][k]))/U[i][i]
    return L,U

In [375]:
A = np.array([[4, -2, 1], [20, -7, 12], [-8, 13, 17]])

L,U = LU_decomposition(A)
print(' L: \n', L, '\n U: \n', U)

 L: 
 [[ 1.  0.  0.]
 [ 5.  1.  0.]
 [-2.  3.  1.]] 
 U: 
 [[ 4. -2.  1.]
 [ 0.  3.  7.]
 [ 0.  0. -2.]]


## Punto 9. Método de Jacobi: diagonalización de matrices simétricas.

#### (a) Implemente el método de Jacobi para encontrar los valores y vectores propios de $\mathbb{A}$.

In [376]:
def jacobiDiagonalization(A, itmax=1000, tol=1e-14):
    
    n = A.shape[0]
    D = A.copy()
    VMatrix = np.identity(n)
    
    # SUMA DE LOS ELEMENTOS DISTINTOS A LA DIAGONAL
    # Y LOS ELEMENTOS EN EL TRIANGULO INFERIOR
    res = np.sum(np.abs(np.triu(D, k=1)))
    
    count = 0

    while count<itmax and res>tol:
        # MATRIZ TRIANGULAR SUPERIOR
        DUP = np.triu(D, k=1)
        # INDICE MAYOR EN LA DIAGONAL
        idx = np.argmax(np.abs(DUP))
        p, q = np.unravel_index(idx, DUP.shape)
        
        
        # MATRIZ DE ROTACIÓN R(THETA)
        theta = np.arctan2(2.*D[p, q],(D[q, q] - D[p, p]))/2.
        R = np.identity(n)
        R[p, p] = np.cos(theta)
        R[q, q] = np.cos(theta)
        R[p, q] = -np.sin(theta)
        R[q, p] = np.sin(theta)

        # ACTUALIZACIÓN MATRIZ DE VALORES Y VECTORES PROPIOS
        D = R.T @ D @ R
        VMatrix = VMatrix @ R

        # SUMAR LOS VALORES QUE QUEDARON POR FUERA DE LA DIAGONAL
        res = np.sum(np.abs(np.triu(D, k=1)))
        count += 1

    eigValues = np.diag(D)
    eigVectors = VMatrix / np.linalg.norm(VMatrix, axis=0)
    eigVectors[:,1] = -eigVectors[:,1]
    
    return eigValues, eigVectors

In [377]:
A = np.array([[4,1,1], [1,3,2], [1,2,5]])

print('Con el método de Jacobi:\n\n','- Valores propios: \n', jacobiDiagonalization(A)[0]
      ,'\n\n - Matriz de vectores propios (columna): \n', jacobiDiagonalization(A)[1])

Con el método de Jacobi:

 - Valores propios: 
 [6.8672772  3.39725929 1.73546351] 

 - Matriz de vectores propios (columna): 
 [[ 0.44012262  0.88613215  0.14512713]
 [ 0.43290596 -0.06779913 -0.89888582]
 [ 0.78669213 -0.45844638  0.41345182]]


#### (b) Compare con el resultado que se obtiene de Numpy: np.linalg.eig(A).

In [378]:
print('Con la función predeterminada de numpy:\n\n','- Valores propios: \n', np.linalg.eig(A)[0]
      ,'\n\n - Matriz de vectores propios (columna): \n', np.linalg.eig(A)[1])

Con la función predeterminada de numpy:

 - Valores propios: 
 [6.89510652 3.39729507 1.70759841] 

 - Matriz de vectores propios (columna): 
 [[ 0.43170413  0.88573564  0.17059871]
 [ 0.49725362 -0.07589338 -0.86427949]
 [ 0.75257583 -0.45794385  0.47319874]]


In [379]:
print('Diferencia valores propios: \n', np.linalg.eig(A)[0]-jacobiDiagonalization(A)[0],
     '\n\nDiferencia vectores propios: \n', np.linalg.eig(A)[1]-jacobiDiagonalization(A)[1])

Diferencia valores propios: 
 [ 2.78293166e-02  3.57800416e-05 -2.78650967e-02] 

Diferencia vectores propios: 
 [[-0.00841849 -0.00039651  0.02547158]
 [ 0.06434766 -0.00809426  0.03460633]
 [-0.0341163   0.00050253  0.05974692]]


## Punto 10. Quantum system - ground state.

In [481]:
H = np.array([[1,2,-1], [1,0,1], [4,-4,5]])

## Punto 13.  (Group theory, Sympy)

In [493]:
Jx = sym.Matrix([[0, 0, 0], [0, 0, -1], [0, 1, 0]])
Jy = sym.Matrix([[0, 0, 1], [0, 0, 0], [-1, 0, 0]])
Jz = sym.Matrix([[0, -1, 0], [1, 0, 0], [0, 0, 0]])

**Caso 1.** [$J_i,J_j$] = $\epsilon_ij J_k$ para $i=x$, $j=y$ y $j=x$, $i=y$:

In [495]:
Jx@Jy-Jy@Jx - 1*Jz

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

In [496]:
Jy@Jx-Jx@Jy - (-1)*Jz

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

Verificando así que la igualdad se cumple.

**Caso 2.** [$J_i,J_j$] = $\epsilon_ij J_k$ para $i=x$, $j=z$ y $j=x$, $i=z$::

In [500]:
Jz@Jx-Jx@Jz - 1*Jy

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

In [501]:
Jx@Jz-Jz@Jx - (-1)*Jy

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

**Caso 3.** [$J_i,J_j$] = $\epsilon_ij J_k$ para $i=y$, $j=z$ y $j=y$, $i=z$::

In [510]:
Jz@Jy-Jy@Jz - (-1)*Jx

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

In [511]:
Jy@Jz-Jz@Jy - 1*Jx

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

**Caso 3.** [$J_i,J_j$] = $\epsilon_ij J_k = 0$ para indices repetidos:

In [514]:
Jx@Jx-Jx@Jx

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

In [515]:
Jy@Jy-Jy@Jy

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

In [516]:
Jz@Jz-Jz@Jz

⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦

Verificando así que la igualdad se cumple para todos los casos.

## Punto 14. (Group theory, Sympy)

In [519]:
sigmax = sym.Matrix([[0, 1], [1, 0]])
sigmay = sym.Matrix([[0, -1j], [1j, 0]])
sigmaz = sym.Matrix([[1, 0], [0, -1]])

Verifiquemos ahroa que se satisface que [$\sigma_i, \sigma_j$] $= 2i\epsilon_{ijk}\sigma_{k}$:

In [523]:
def conmutador(A, B):
    return A@B-B@A

Caso 1.1. $i=x$ y $j=y$.

In [546]:
conmutador(sigmax,sigmay) - 2*(1)*1j*sigmaz

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

Caso 1.2. $i=y$ y $j=x$.

In [547]:
conmutador(sigmay,sigmax) + 2*(1)*1j*sigmaz

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

Caso 2.1. $i=x$ y $j=z$.

In [548]:
conmutador(sigmax,sigmaz) - 2*(-1)*1j*sigmay

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

Caso 2.2. $i=z$ y $j=x$.

In [549]:
conmutador(sigmaz,sigmax) + 2*(-1)*1j*sigmay

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

Caso 3.1. $i=y$ y $j=z$.

In [550]:
conmutador(sigmay,sigmaz) - 2*(1)*1j*sigmax

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

Caso 3.2. $i=z$ y $j=y$.

In [551]:
conmutador(sigmaz,sigmay) + 2*(1)*1j*sigmax

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

Caso 4. Indices repetidos.

## Punto 15. (Particle Physics, Sympy)

In [174]:
gamma0 = sym.Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]])
gamma1 = sym.Matrix([[0, 0, 0, 1], [0, 0, 1, 0], [0, -1, 0, 0], [-1, 0, 0, 0]])
gamma2 = sym.Matrix([[0, 0, 0, -1j], [0, 0, 1j, 0], [0, 1j, 0, 0], [-1j, 0, 0, 0]])
gamma3 = sym.Matrix([[0, 0, 1, 0], [0, 0, 0, -1], [-1, 0, 0, 0], [0, 1, 0, 0]])

Para todo anticonmutador con indices $\nu$, $\mu$ distintos se tiene una matriz de ceros. Tenga en cuenta las entradas de la relación del anticonmutador

$$ \lbrace\gamma^{\nu},\gamma^{\mu}\rbrace = \lbrace\gamma^{\mu},\gamma^{\nu}\rbrace  $$

ya que se tiene conmutatividad bajo suma. Entonces, todos los posibles casos para la matriz del anticonmutador distintas a la diagonal están dadas por:

In [175]:
gamma0@gamma1+gamma1@gamma0, gamma0@gamma2+gamma2@gamma0, gamma0@gamma3+gamma3@gamma0, gamma1@gamma2+gamma2@gamma1, gamma1@gamma3+gamma3@gamma1, gamma2@gamma3+gamma3@gamma2

⎛⎡0  0  0  0⎤  ⎡0  0  0  0⎤  ⎡0  0  0  0⎤  ⎡0  0  0  0⎤  ⎡0  0  0  0⎤  ⎡0  0  
⎜⎢          ⎥  ⎢          ⎥  ⎢          ⎥  ⎢          ⎥  ⎢          ⎥  ⎢      
⎜⎢0  0  0  0⎥  ⎢0  0  0  0⎥  ⎢0  0  0  0⎥  ⎢0  0  0  0⎥  ⎢0  0  0  0⎥  ⎢0  0  
⎜⎢          ⎥, ⎢          ⎥, ⎢          ⎥, ⎢          ⎥, ⎢          ⎥, ⎢      
⎜⎢0  0  0  0⎥  ⎢0  0  0  0⎥  ⎢0  0  0  0⎥  ⎢0  0  0  0⎥  ⎢0  0  0  0⎥  ⎢0  0  
⎜⎢          ⎥  ⎢          ⎥  ⎢          ⎥  ⎢          ⎥  ⎢          ⎥  ⎢      
⎝⎣0  0  0  0⎦  ⎣0  0  0  0⎦  ⎣0  0  0  0⎦  ⎣0  0  0  0⎦  ⎣0  0  0  0⎦  ⎣0  0  

0  0⎤⎞
    ⎥⎟
0  0⎥⎟
    ⎥⎟
0  0⎥⎟
    ⎥⎟
0  0⎦⎠

Luego, las entradas distintas a la diagonal, es decir $\lbrace\gamma^{\nu},\gamma^{\mu}\rbrace = 0$ para $\nu\neq\mu$.


Por otro lado, las entradas diagonales están dadas por:

In [177]:
gamma0@gamma0+gamma0@gamma0, gamma1@gamma1+gamma1@gamma1, gamma2@gamma2+gamma2@gamma2, gamma3@gamma3+gamma3@gamma3

⎛⎡2  0  0  0⎤  ⎡-2  0   0   0 ⎤  ⎡-2.0   0     0     0  ⎤  ⎡-2  0   0   0 ⎤⎞
⎜⎢          ⎥  ⎢              ⎥  ⎢                      ⎥  ⎢              ⎥⎟
⎜⎢0  2  0  0⎥  ⎢0   -2  0   0 ⎥  ⎢ 0    -2.0   0     0  ⎥  ⎢0   -2  0   0 ⎥⎟
⎜⎢          ⎥, ⎢              ⎥, ⎢                      ⎥, ⎢              ⎥⎟
⎜⎢0  0  2  0⎥  ⎢0   0   -2  0 ⎥  ⎢ 0     0    -2.0   0  ⎥  ⎢0   0   -2  0 ⎥⎟
⎜⎢          ⎥  ⎢              ⎥  ⎢                      ⎥  ⎢              ⎥⎟
⎝⎣0  0  0  2⎦  ⎣0   0   0   -2⎦  ⎣ 0     0     0    -2.0⎦  ⎣0   0   0   -2⎦⎠

Por los que las entradas de la diagonal están dadas por:
$$\lbrace\gamma^{0},\gamma^{0}\rbrace = 2\cdot \mathbb{I}_{4\times 4},$$
$$\lbrace\gamma^{1},\gamma^{1}\rbrace = -2\cdot \mathbb{I}_{4\times 4},$$
$$\lbrace\gamma^{2},\gamma^{2}\rbrace = -2\cdot \mathbb{I}_{4\times 4},$$
$$\lbrace\gamma^{3},\gamma^{3}\rbrace = -2\cdot \mathbb{I}_{4\times 4}.$$

Luego, 
$$\lbrace\gamma^{\mu},\gamma^{\nu}\rbrace = \left[\begin{array}{cccc}
2 & 0 & 0 & 0	\\
0 & -2 & 0 & 0	\\
0 & 0 & -2 & 0	\\
0 & 0 & 0 & -2
\end{array}\right] = 2\eta^{\mu\nu}\mathbb{I}_{4\times 4}$$


para $\mu,\nu$ que toman los valores de $0,1,2,3$ y $\eta^{\mu\nu}=diag(+1,-1,-1,-1)$.

Verificando así que
$$\lbrace\gamma^{\mu},\gamma^{\nu}\rbrace = \gamma^{\mu}\gamma^{\nu} + \gamma^{\nu}\gamma^{\mu} = 2\eta^{\mu\nu}\mathbb{I}_{4\times 4}.$$